# Demonstration!

Instructions for installing hepconvert are here 

In [8]:
# First import hepconvert (or to install; 

import hepconvert
import uproot
import awkward as ak

# Copy ROOT Files



# Sum histograms



# Go back and forth?